# Project 2
Lab journal

Bogdan Sotnikov

## Day 1 20.11.22

### Step 1. The preparing to the investigation.

We created virtual environment for this project.

```bash
conda create -n BI_proj3
```

We downloaded three complects of paired reads (by one forward and reverse  in each complect).

We used bash command for downloading this data

```bash
wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R1_001.fastq.gz 

wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R2_001.fastq.gz 

wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R1_001.fastq.gz 

wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R2_001.fastq.gz 

wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R1_001.fastq.gz 

wget https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R2_001.fastq.gz 

```
The first pair comtained paired-end reads and the second two pairs contained mate-paired reads.

### Step 2. The data analysis.

We used fastqc (v0.11.9) to check our data's quality. Quality was good, without any abnormalities.

```bash
fastqc *_001.fastq.gz 
```

For k-mer analyzing we used jellyfish (v. 2.3.0).
We used it with recomended options: m = 31 (number of k-mers), -C for ignoring strands and 10000000 for nu,ber of entries.

```bash
jellyfish count -m 31 -o jfoutput_1 -s 10000000 -C SRR292678sub_S1_L001_R1_001.fastq SRR292678sub_S1_L001_R2_001.fastq
```
Then we found more detailed explanation for working with jelyyfish and created another file.

```bash
jellyfish count -m 31 -o jfoutput_31 -s 50000000 -C --min-quality=20 --quality-start=33 SRR292678sub*
```

We added options obout quality-coding (PHRED-33) and about threshold for quality.

Then we used jellyfish histo for creating human-readable data about k-mers.

On the next stage we visualised k-mer distribution in R (v. 4.2.1).

```r
setwd("<path_to_directory>")
library(ggplot2)
spec1_31 <- read.table("jfoutput_31.histo")
plot(spec1_31[2:200,],type="l")

ggplot(spec1_31[2:200,])+
  geom_point(aes(x=spec1_31[2:200,]$V1, y=spec1_31[2:200,]$V2))+
  xlab(label = "Frequency of k-mer")+
  ylab(label = "Frequency of k-mer's frequency")

total_kmer <- sum(as.numeric(spec1_31[3:nrow(spec1_31), 1]*spec1_31[3:nrow(spec1_31), 2]))
total_kmer
peak_kmer <- total_kmer/48
peak_kmer


# 269318297
# 5610798
```

!!!Вставить картинку!!!

We found inflection point (3), total number of k-mers (269318297) and peak position (48). We can compute genome size as  T/(M*L)/(L-K+1), where M: Kmer peak, K: Kmer-size, L: avg read length T: Total bases. Divider is coverage depth.
Genome size is equal (5499346*90)/(48*90)/(90-31+1) = 6874183 = 6.9 G.
Another genome size equation is total number of k-mers/peak position = 5.6 G. Second estimation looks more realistically.

 and genome size (5610798 bases = 5.6 Gb).

### Step 3. Genome assembling.

#### Substep 3.1. 1-library assembling.

For assembling E. coli X genome from paired reads we used SPAdes (v3.15.5).

I tried to start spades with below-writed options:

```bash
spades.py --isolate --pe1-1 SRR292678sub_S1_L001_R1_001.fastq --pe2-1 SRR292678sub_S1_L001_R2_001.fastq -o <path_to_directory>
```

Spades didn't work with these options. Some time later I found one mistake: I wrote "--pe2-1" in place of "--pe1-2". When I corrected it, process had started.

The correct command was 

```bash
spades.py --isolate --pe1-1 SRR292678sub_S1_L001_R1_001.fastq --pe1-2 SRR292678sub_S1_L001_R2_001.fastq -o <path_to_directory>
```

But it wasn't finally correct too, beacuse I printed path from "/" and spades created new path. I had needed to move files to working directory (and I did it).

For assesing quality of assembling we used QUAST (v. 2.31). Quast was uncompartible with python 3.11, thats why we created new conda environment.

```bash
conda create -n BI_proj39 python=3.9
```
We started this tool with below-described options.

```bash
quast.py contigs.fasta -o ~/bioinf/ngs/proj3
```

Information from output was 
```
All statistics are based on contigs of size >= 500 bp, unless otherwise noted (e.g., "# contigs (>= 0 bp)" and "Total length (>= 0 bp)" include all contigs).

Assembly                    contigs 
# contigs (>= 0 bp)         885     
# contigs (>= 1000 bp)      127     
# contigs (>= 5000 bp)      86      
# contigs (>= 10000 bp)     73      
# contigs (>= 25000 bp)     54      
# contigs (>= 50000 bp)     32      
Total length (>= 0 bp)      5363438 
Total length (>= 1000 bp)   5227528 
Total length (>= 5000 bp)   5112460 
Total length (>= 10000 bp)  5024868 
Total length (>= 25000 bp)  4704658 
Total length (>= 50000 bp)  3898474 
# contigs                   162     
Largest contig              300784  
Total length                5252199 
GC (%)                      50.54   
N50                         114227  
N90                         24915   
auN                         126907.1
L50                         15      
L90                         55      
# N's per 100 kbp           0.00  
```

It was slight different data for scaffolds:

```
All statistics are based on contigs of size >= 500 bp, unless otherwise noted (e.g., "# contigs (>= 0 bp)" and "Total length (>= 0 bp)" include all contigs).

Assembly                    scaffolds
# contigs (>= 0 bp)         868      
# contigs (>= 1000 bp)      123      
# contigs (>= 5000 bp)      83       
# contigs (>= 10000 bp)     70       
# contigs (>= 25000 bp)     53       
# contigs (>= 50000 bp)     32       
Total length (>= 0 bp)      5365096  
Total length (>= 1000 bp)   5228807  
Total length (>= 5000 bp)   5115861  
Total length (>= 10000 bp)  5026097  
Total length (>= 25000 bp)  4751300  
Total length (>= 50000 bp)  3989113  
# contigs                   168      
Largest contig              300784   
Total length                5260240  
GC (%)                      50.52    
N50                         114589   
N90                         25454    
auN                         130855.3 
L50                         14       
L90                         53       
# N's per 100 kbp           32.47 
```

#### Substep 3.2. 3-library assembling.

For assembling with matepairs we used below-writed code:

```bash
spades.py --isolate --pe1-1 SRR292678sub_S1_L001_R1_001.fastq.gz --pe1-2 SRR292678sub_S1_L001_R2_001.fastq.gz --mp1-1 SRR292770_S1_L001_R2_001.fastq.gz --mp1-2 SRR292770_S1_L001_R1_001.fastq.gz --mp2-1 SRR292862_S2_L001_R2_001.fastq.gz --mp2-2 SRR292862_S2_L001_R1_001.fastq.gz -o <path_for_output>
```

We assessed quality of assembling by quast:

```bash
quast.py contigs.fasta -o <path_for_output>
```

```bash
quast.py scaffolds.fasta -o <path_for_output>
```

with two outputs respectively:

```
Assembly                    contigs 
# contigs (>= 0 bp)         1154    
# contigs (>= 1000 bp)      126     
# contigs (>= 5000 bp)      66      
# contigs (>= 10000 bp)     54      
# contigs (>= 25000 bp)     44      
# contigs (>= 50000 bp)     28      
Total length (>= 0 bp)      5402541 
Total length (>= 1000 bp)   5221609 
Total length (>= 5000 bp)   5090134 
Total length (>= 10000 bp)  5011459 
Total length (>= 25000 bp)  4846400 
Total length (>= 50000 bp)  4280675 
# contigs                   169     
Largest contig              391919  
Total length                5251556 
GC (%)                      50.54   
N50                         157905  
N90                         29147   
auN                         177788.0
L50                         11      
L90                         40      
# N's per 100 kbp           0.00
```

```
Assembly                    scaffolds
# contigs (>= 0 bp)         1070     
# contigs (>= 1000 bp)      56       
# contigs (>= 5000 bp)      13       
# contigs (>= 10000 bp)     12       
# contigs (>= 25000 bp)     11       
# contigs (>= 50000 bp)     11       
Total length (>= 0 bp)      5531597  
Total length (>= 1000 bp)   5351356  
Total length (>= 5000 bp)   5266301  
Total length (>= 10000 bp)  5259820  
Total length (>= 25000 bp)  5248736  
Total length (>= 50000 bp)  5248736  
# contigs                   108      
Largest contig              1046981  
Total length                5386995  
GC (%)                      50.52    
N50                         741159   
N90                         426441   
auN                         719107.4 
L50                         3        
L90                         7        
# N's per 100 kbp           2395.70 
```

## Day 2. 21.11.22

Step 4. Annotating.

For annotating assembled genome we used prokka (v. 2.31). It were some problems with installation, but, as it looks, now all is coreect. (UPD - it didn't works).

I looked log and found that orikka conflicted with QUAST. Then I created new conda environment.

```bash
conda install -c bioconda prokka
```
There were no warnings, but prokka didn't works too.

Then I tried a next way:

```bash
sudo apt-get install libdatetime-perl libxml-simple-perl libdigest-md5-perl git default-jre bioperl
sudo cpan Bio::Perl
git clone https://github.com/tseemann/prokka.git $HOME/prokka
$HOME/prokka/bin/prokka --setupdb
```
New attempt leaded to new failure.

We used the next script

```bash
sudo apt-get -y update
sudo apt-get install bioperl libxml-simple-perl default-jre git curl



git clone https://github.com/tseemann/prokka.git
export PATH=$PWD/prokka/bin:$PATH
prokka --setupdb
prokka --version
```

We used prokka FROM WORKING DIRECTORY, WHERE "PROKKA" FOLDER WAS SITUATED (sorry for caps) with below-writed script.

```bash
git clone https://github.com/tseemann/prokka.git
export PATH=$PWD/prokka/bin:$PATH
prokka --setupdb
prokka --version
```

BUT IT DIDN'T WORK(sorry for caps)

I created new one venv.

```bash
conda create -n prokka_env -c conda-forge -c bioconda prokka
```

And started prokka (v. 1.14.6)

```bash
prokka ./spades_3library/scaffolds.fasta
```


### Step 5. Finding 16S.

For finding 16S (we need them to estimate more relative E.coli strains) we used barrnap (v 0.9). We entered this tool with default parameters.

```bash
barrnap scaffolds.fasta > 16srna
```

With the output:

```
##gff-version 3
NODE_1_length_1046981_cov_69.027703	barrnap:0.9	rRNA	405650	405760	2.3e-11	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_266_length_223_cov_0.720238	barrnap:0.9	rRNA	19	129	5.7e-18	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_2_length_981131_cov_67.425569	barrnap:0.9	rRNA	526837	528374	0-	.	Name=16S_rRNA;product=16S ribosomal RNA
NODE_3_length_741159_cov_70.451946	barrnap:0.9	rRNA	353641	355178	0+	.	Name=16S_rRNA;product=16S ribosomal RNA
NODE_56_length_1031_cov_481.502049	barrnap:0.9	rRNA	9	1026	5.9e-258	-	.	Name=23S_rRNA;product=23S ribosomal RNA (partial);note=aligned only 31 percent of the 23S ribosomal RNA
NODE_5_length_546617_cov_64.838179	barrnap:0.9	rRNA	124551	124661	2.3e-11	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_5_length_546617_cov_64.838179	barrnap:0.9	rRNA	221914	222024	2.3e-11	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_5_length_546617_cov_64.838179	barrnap:0.9	rRNA	361249	361359	2.3e-11	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_5_length_546617_cov_64.838179	barrnap:0.9	rRNA	402585	402695	2.3e-11	+	.	Name=5S_rRNA;product=5S ribosomal RNA
NODE_76_length_721_cov_0.948949	barrnap:0.9	rRNA	2	407	1e-22	-.	Name=16S_rRNA;product=16S ribosomal RNA (partial);not
```

For more detailed output I used another command:

```bash
barrnap -o rrna.fa < scaffolds.fasta > rrna.gff
```

Then we ran two 16s intervals in blastn with params "Entrez Query":1900/01/01:2011/01/01[PDAT].

The two nearest organisms were Escherichia coli 55989 chromosome and Shigella flexneri 2a str. 2457T for first 16S sequence; Escherichia coli O26:H11 str. 11368,  	Escherichia coli 55989, Escherichia coli HS,  	Shigella dysenteriae strain FBD015, Shigella sonnei Ss046 for second 16s sequence; Wallemia ichthyophaga strain CBS 113033 for third 16S sequence.

We thank most probable variant is Escherichia coli 55989. The NCBI ID of this bactery is NCBI:txid585055


## Day 3. 25.11.22
### Step 6. Comparing two E.Coli sequences.

I created new virtual environment.

```bash
conda create -n mauve
```

Downloaded mauve and old java version.

```bash
conda install mauve
sudo apt install openjdk-8-jdk 
```


We used Mauve (v. ) for this purpose. I added reference file and anotation made by prokka and searching products by pattern "toxi".

We add reference file 55989.fasta and annotation by prokka.

We found shiga related toxins, which were in E. coli X annotation, not in reference. It may cause hemorragic properties of E.coli X. Data about shiga toxin coding parts is below-listed.

5307965 5308234 stxB (код на uniprot Q7BQ98 - можно вставить модельку в статью).

5308246 5309245 stxA (Q9FBI2)

Another interesting insertions:
toxic protein HokE 5441456 5441668 (P77091)

3493013 3493390 P76364

LtxB 4488967 4491069 P23702 (пришла из какого-то аггрибактера).

Toxin CCDB 5155511 5155816 P62554

Also we have many transposase proteins in E. coli X, which means there are many transposons in it.

I have seen many hypothetical proteins, so I used .gbk annotation from project manual.

Findings

Enterotoxin with long name: 3016232 3019567

Another one shiga toxin 3483605 3483874

3483886 3484845

VapC 5219438 5219866

Doc toxin 5224169 5224444

## Day 4. 26.11.22
### Step 7. Finding antibiotic resistance genes.

For finding resistance genes I used [ResFinder website](https://cge.food.dtu.dk/services/ResFinder/) v. 4.1 with default options (threshold for % ID - 90%, minimum length - 60%, type of the data - assembled genome/contigs)

It is resistant to ampicillin, sulfamethoxazole, trimethoprim, cephalothin
piperacillin, tetracycline, amoxicillin, ethidium bromide, cefepime, doxycycline, benzylkonium chloride, ceftriaxone, ceftazidime, ticarcillin, chlorhexidine, cetylpyridinium chloride, cefotaxime, aztreonam  and streptomycin.

By comparison reference sequence has resistance only for doxycycline, tetracycline and minocycline.

Results means that besides tetracycline resistance E.coli X has resistance for amny beta-lactams, folate pathway antagonists and some antiseptics (quaternary ammonium compounds).

I started to find genes, causes this new resistance mechanisms. I used mauve with same parameters, as in previous step.

We found beta-lactamase gene TEM (which was absent in reference) in 5108214-5109083 (P62593), and CTX-M1 in 51111970-5112845 (P28585).

Also I found new tetracycline resistance gene, which was absent too in reference -- tetA 5281235-5282434 (P02981).

We need some additional information to interpret significanse of presence dhfrI (dihydrofolate reductase type 1) 4957362-5049029 (P00382).


I found transposone resolvase and transposase near two bla-genes. It may account that this genes came from transposones.